In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
import sys

sys.path.insert(0, '../..')

## Load Dataset

In [4]:
from entity_embed.benchmarks import DBLP_ACM_StructuredBenchmark

benchmark = DBLP_ACM_StructuredBenchmark(data_dir_path="../data/")
benchmark

17:14:47 INFO:Extracting DBLP-ACM-Structured...
17:14:47 INFO:Reading DBLP-ACM-Structured row_dict...
17:14:47 INFO:Reading DBLP-ACM-Structured train.csv...
17:14:47 INFO:Reading DBLP-ACM-Structured valid.csv...
17:14:47 INFO:Reading DBLP-ACM-Structured test.csv...


<DBLP_ACM_StructuredBenchmark> from http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/DBLP-ACM/dblp_acm_exp_data.zip

## Preprocess

In [5]:
attr_list = ['title', 'authors', 'venue', 'year']

In [6]:
from tqdm.auto import tqdm
import unidecode
import itertools
from entity_embed import default_tokenizer

def clean_str(s):
    s = unidecode.unidecode(s).lower().strip()
    s_tokens = itertools.islice((s_part[:30] for s_part in default_tokenizer(s)), 0, 30)
    return ' '.join(s_tokens)[:300]

for row in tqdm(benchmark.row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

## Init Data Module

In [7]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [8]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [9]:
attr_info_dict = {
    'title': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'use_mask': True,
        'max_str_len': None,  # compute
    },
    'semantic_title': {
        'source_attr': 'title',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
        'use_mask': True,
    },
    'authors': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'use_mask': True,
        'max_str_len': None,  # compute
    },
    'venue': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'use_mask': True,
        'max_str_len': None,  # compute
    },
    'semantic_venue': {
        'source_attr': 'venue',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
        'use_mask': True,
    },
    'year': {
        'field_type': "STRING",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    }
}

In [10]:
from entity_embed import AttrInfoDictParser

row_numericalizer = AttrInfoDictParser.from_dict(attr_info_dict, row_dict=benchmark.row_dict)
row_numericalizer.attr_info_dict

17:14:47 INFO:For attr=title, computing actual max_str_len
17:14:47 INFO:For attr=title, using actual_max_str_len=18
17:14:47 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:14:51 INFO:For attr=authors, computing actual max_str_len
17:14:51 INFO:For attr=authors, using actual_max_str_len=18
17:14:51 INFO:For attr=venue, computing actual max_str_len
17:14:51 INFO:actual_max_str_len=13 must be pair to enable NN pooling. Updating to 14
17:14:51 INFO:For attr=venue, using actual_max_str_len=14
17:14:51 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:14:54 INFO:For attr=year, computing actual max_str_len
17:14:54 INFO:For attr=year, using actual_max_str_len=4


{'title': NumericalizeInfo(source_attr='title', field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=18, vocab=None, n_channels=8, embed_dropout_p=0.2, use_attention=True, use_mask=True),
 'semantic_title': NumericalizeInfo(source_attr='title', field_type=<FieldType.SEMANTIC_MULTITOKEN: 'semantic_multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 

In [11]:
datamodule = benchmark.build_pairwise_datamodule(
    row_numericalizer=row_numericalizer,
    batch_size=20,
    row_batch_size=16,
    random_seed=random_seed
)

## Training

In [12]:
from entity_embed import LinkageEmbed

ann_k = 100
model = LinkageEmbed(
    datamodule,
    ann_k=ann_k,
    embedding_size=300
)

In [13]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 100
early_stop_callback = EarlyStopping(
   monitor='valid_f1_at_0.7',
   min_delta=0.00,
   patience=20,
   verbose=True,
   mode='max'
)
tb_log_dir = '../tb_logs'
tb_name = f'f1-{benchmark.dataset_name}'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_log_dir, name=tb_name)
)

17:14:54 INFO:GPU available: True, used: True
17:14:54 INFO:TPU available: None, using: 0 TPU cores
17:14:54 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [14]:
trainer.fit(model, datamodule)

17:14:56 INFO:
  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 7.6 M 
1 | losser      | SupConLoss | 0     
-------------------------------------------
6.5 M     Trainable params
1.1 M     Non-trainable params
7.6 M     Total params


1

In [15]:
model.blocker_net.get_signature_weights()

{'title': 0.2641853392124176,
 'semantic_title': 0.2497837394475937,
 'authors': 0.24568788707256317,
 'venue': 0.09253549575805664,
 'semantic_venue': 0.0541272833943367,
 'year': 0.0936802327632904}

In [16]:
from entity_embed import validate_best

validate_best(trainer)

{'valid_f1_at_0.3': 0.3340857787810384,
 'valid_f1_at_0.5': 0.9124613800205973,
 'valid_f1_at_0.7': 0.9854096520763187,
 'valid_f1_at_0.9': 0.9769585253456221,
 'valid_pair_entity_ratio_at_0.3': 2.4932432432432434,
 'valid_pair_entity_ratio_at_0.5': 0.5934684684684685,
 'valid_pair_entity_ratio_at_0.7': 0.5033783783783784,
 'valid_pair_entity_ratio_at_0.9': 0.4774774774774775,
 'valid_precision_at_0.3': 0.2005420054200542,
 'valid_precision_at_0.5': 0.8406072106261859,
 'valid_precision_at_0.7': 0.9821029082774049,
 'valid_precision_at_0.9': 1.0,
 'valid_recall_at_0.3': 1.0,
 'valid_recall_at_0.5': 0.9977477477477478,
 'valid_recall_at_0.7': 0.9887387387387387,
 'valid_recall_at_0.9': 0.954954954954955}

## Testing

In [17]:
trainer.test(ckpt_path='best', verbose=False)

[{'test_f1_at_0.3': 0.3852494577006508,
  'test_f1_at_0.5': 0.9357218124341412,
  'test_f1_at_0.7': 0.9843400447427293,
  'test_f1_at_0.9': 0.9781357882623705,
  'test_pair_entity_ratio_at_0.3': 2.095720720720721,
  'test_pair_entity_ratio_at_0.5': 0.5686936936936937,
  'test_pair_entity_ratio_at_0.7': 0.5067567567567568,
  'test_pair_entity_ratio_at_0.9': 0.4786036036036036,
  'test_precision_at_0.3': 0.23858140784524448,
  'test_precision_at_0.5': 0.8792079207920792,
  'test_precision_at_0.7': 0.9777777777777777,
  'test_precision_at_0.9': 1.0,
  'test_recall_at_0.3': 1.0,
  'test_recall_at_0.5': 1.0,
  'test_recall_at_0.7': 0.990990990990991,
  'test_recall_at_0.9': 0.9572072072072072}]